# LlamaIndexでWorkflows

## 目次
- [概要](#概要)
- [参考](#参考)
- [チェック](#チェック)
- [準備](#準備)
- [最も簡単なエージェント](#最も簡単なエージェント)
- [エージェントにRAGを追加する](#エージェントにRAGを追加する)

## 概要
- LlamaIndex（公式）をトレースして基本的な利用方法を確認する。
- 破壊的に変更が発生するまで使えるでしょう。
- 破壊的に変更が発生後は、公式サイトの当該バージョンの情報（≒一次情報）をあたって。

## 参考

LlamaIndex - .NET 開発基盤部会 Wiki  
https://dotnetdevelopmentinfrastructure.osscons.jp/index.php?LlamaIndex

## チェック

In [1]:
#!pip list

In [2]:
#%env

## 準備

### インストレーション

```Python
!pip install llama-index-utils-workflow
```

### ライブラリ読み込み

In [3]:
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    StartEvent,
    StopEvent,
    Context,
)
from llama_index.utils.workflow import draw_all_possible_flows

In [4]:
from IPython.display import IFrame, display

def display_html(file_path = "example.html"):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()
    display(IFrame(file_path, width="100%", height="300"))

### LLMの設定

#### OpenAI

In [5]:
from llama_index.llms.openai import OpenAI
#llm = OpenAI(model="gpt-3.5-turbo", temperature=0)

#### Ollama

In [6]:
from llama_index.llms.ollama import Ollama
#llm = Ollama(model="Llama3", temperature=0, request_timeout=360.0)

## シングルステップ

### ワークフロー定義

In [7]:
class MyWorkflow(Workflow):
    @step
    async def my_step(self, ev: StartEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")

### ワークフロー実行

In [8]:
w = MyWorkflow(timeout=10, verbose=False)
result = await w.run()
print(result)

Hello, world!


## ループと分岐

### ループ

#### カスタム・イベント

In [9]:
class LoopEvent(Event):
    loop_output: str
class FirstEvent(Event):
    first_output: str

#### ワークフロー定義

In [10]:
import random

class LoopWorkflow(Workflow):
    @step
    async def step_one(self, ev: StartEvent | LoopEvent) -> FirstEvent | LoopEvent:
        if random.randint(0, 1) == 0:
            print("Bad thing happened")
            return LoopEvent(loop_output="Back to step one.")
        else:
            print("Good thing happened")
            return FirstEvent(first_output="First step complete.")

    @step
    async def first_step(self, ev: FirstEvent) -> StopEvent:
        # do something here
        return StopEvent(result="Hello, world!")

#### ワークフロー表示

In [11]:
filename="./llamaindex/LoopWorkflow.html"
draw_all_possible_flows(LoopWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.FirstEvent'>
<class '__main__.LoopEvent'>
./llamaindex/LoopWorkflow.html


#### ワークフロー実行

In [12]:
w = LoopWorkflow()
result = await w.run()
print(result)

Good thing happened
Hello, world!


### 分岐

#### カスタム・イベント

In [13]:
class BranchA1Event(Event):
    payload: str
class BranchA2Event(Event):
    payload: str
class BranchB1Event(Event):
    payload: str
class BranchB2Event(Event):
    payload: str

#### ワークフロー定義

In [14]:
import random

class BranchWorkflow(Workflow):
    @step
    async def start(self, ev: StartEvent) -> BranchA1Event | BranchB1Event:
        if random.randint(0, 1) == 0:
            print("Go to branch A")
            return BranchA1Event(payload="Branch A")
        else:
            print("Go to branch B")
            return BranchB1Event(payload="Branch B")

    @step
    async def step_a1(self, ev: BranchA1Event) -> BranchA2Event:
        print(ev.payload)
        return BranchA2Event(payload=ev.payload)

    @step
    async def step_b1(self, ev: BranchB1Event) -> BranchB2Event:
        print(ev.payload)
        return BranchB2Event(payload=ev.payload)

    @step
    async def step_a2(self, ev: BranchA2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch A complete.")

    @step
    async def step_b2(self, ev: BranchB2Event) -> StopEvent:
        print(ev.payload)
        return StopEvent(result="Branch B complete.")

#### ワークフロー表示

In [15]:
filename="./llamaindex/BranchWorkflow.html"
draw_all_possible_flows(BranchWorkflow, filename)
display_html(filename)

<class 'NoneType'>
<class '__main__.BranchA1Event'>
<class '__main__.BranchB1Event'>
<class '__main__.BranchA2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
<class '__main__.BranchB2Event'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/BranchWorkflow.html


#### ワークフロー実行

In [16]:
w = BranchWorkflow()
result = await w.run()
print(result)

Go to branch A
Branch A
Branch A
Branch A complete.


#### カスタム・イベント

In [17]:
class SetupEvent(Event):
    query: str
class StepTwoEvent(Event):
    query: str

#### ワークフロー定義

In [18]:
class StatefulFlow(Workflow):
    @step
    async def setup(self, ctx: Context, ev: SetupEvent) -> StartEvent:
        # load data
        await ctx.set("some_database", [1, 2, 3])
        return StartEvent(query=ev.query)

    @step
    async def start(self, ctx: Context, ev: StartEvent) -> SetupEvent | StepTwoEvent:
        db = await ctx.get("some_database", default=None)
        if db is None:
            print("Need to load data")
            return SetupEvent(query=ev.query)

        # do something with the query
        return StepTwoEvent(query=ev.query)

    @step
    async def step_two(self, ctx: Context, ev: StepTwoEvent) -> StopEvent:
        # do something with the data
        print("Data is ", await ctx.get("some_database"))
    
        return StopEvent(result=await ctx.get("some_database"))

#### ワークフロー表示

In [19]:
filename="./llamaindex/StatefulFlow.html"
draw_all_possible_flows(StatefulFlow, filename)
display_html(filename)

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StartEvent'>
<class '__main__.SetupEvent'>
<class '__main__.StepTwoEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/StatefulFlow.html


#### ワークフロー実行

In [20]:
w = StatefulFlow(timeout=10, verbose=False)
result = await w.run(query="Some query")
print(result)

Need to load data
Data is  [1, 2, 3]
[1, 2, 3]


### ストリーミングイベント

In [21]:
import asyncio

#### カスタム・イベント

In [22]:
class FirstEvent(Event):
    first_output: str
class SecondEvent(Event):
    second_output: str
    response: str
class ProgressEvent(Event):
    msg: str

#### ワークフロー定義

In [23]:
class StreamingWorkflow(Workflow):
    @step
    async def step_one(self, ctx: Context, ev: StartEvent) -> FirstEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step one is happening"))
        return FirstEvent(first_output="First step complete.")

    @step
    async def step_two(self, ctx: Context, ev: FirstEvent) -> SecondEvent:
        llm = OpenAI(model="gpt-3.5-turbo", temperature=0)
        generator = await llm.astream_complete(
            "Please give me the first 3 paragraphs of Moby Dick, a book in the public domain."
        )
        async for response in generator:
            # Allow the workflow to stream this piece of response
            ctx.write_event_to_stream(ProgressEvent(msg=response.delta))
        return SecondEvent(
            second_output="Second step complete, full response attached",
            response=str(response),
        )

    @step
    async def step_three(self, ctx: Context, ev: SecondEvent) -> StopEvent:
        ctx.write_event_to_stream(ProgressEvent(msg="Step three is happening"))
        return StopEvent(result="Workflow complete.")

#### ワークフロー表示

In [24]:
filename="./llamaindex/StreamingWorkflow.html"
draw_all_possible_flows(StatefulFlow, filename)
display_html(filename)

<class 'NoneType'>
<class 'llama_index.core.workflow.events.StartEvent'>
<class '__main__.SetupEvent'>
<class '__main__.StepTwoEvent'>
<class 'llama_index.core.workflow.events.StopEvent'>
./llamaindex/StreamingWorkflow.html


#### ワークフロー実行

##### 非同期関数を定義

In [25]:
async def main():
    w = StreamingWorkflow(timeout=30, verbose=True)
    handler = w.run(first_input="Start the workflow.")

    async for ev in handler.stream_events():
        if isinstance(ev, ProgressEvent):
            print(ev.msg)

    final_result = await handler
    print("Final result", final_result)

##### Pythonのバージョン次第で実行方法方が違う

In [26]:
!python3 -V

Python 3.10.12


In [27]:
!jupyter --version

Selected Jupyter core packages...
IPython          : 8.29.0
ipykernel        : 6.29.5
ipywidgets       : not installed
jupyter_client   : 8.6.3
jupyter_core     : 5.7.2
jupyter_server   : 2.14.2
jupyterlab       : 4.3.1
nbclient         : 0.10.0
nbconvert        : 7.16.4
nbformat         : 5.10.4
notebook         : not installed
qtconsole        : not installed
traitlets        : 5.14.3


##### 非同期関数の実行

In [28]:
#if __name__ == "__main__":
#    asyncio.run(main())
result = await main()

Running step step_one
Step step_one produced event FirstEvent
Running step step_two
Step one is happening

Sure
,
 here
 are
 the
 first
 
3
 paragraphs
 of
 M
oby
 Dick
 by
 Herman
 Mel
ville
:


"
Call
 me
 Ish
ma
el
.
 Some
 years
 ago
—
never
 mind
 how
 long
 precisely
—
having
 little
 or
 no
 money
 in
 my
 purse
,
 and
 nothing
 particular
 to
 interest
 me
 on
 shore
,
 I
 thought
 I
 would
 sail
 about
 a
 little
 and
 see
 the
 wat
ery
 part
 of
 the
 world
.
 It
 is
 a
 way
 I
 have
 of
 driving
 off
 the
 sple
en
 and
 regulating
 the
 circulation
.
 Whenever
 I
 find
 myself
 growing
 grim
 about
 the
 mouth
;
 whenever
 it
 is
 a
 damp
,
 dr
izzly
 November
 in
 my
 soul
;
 whenever
 I
 find
 myself
 invol
unt
arily
 pa
using
 before
 coffin
 warehouses
,
 and
 bringing
 up
 the
 rear
 of
 every
 funeral
 I
 meet
;
 and
 especially
 whenever
 my
 hy
pos
 get
 such
 an
 upper
 hand
 of
 me
,
 that
 it
 requires
 a
 strong
 moral
 principle
 to
 prevent
 me
 from
 delibera